In [3]:
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()
print("Workspace loaded:", ws.name)

Workspace loaded: frauddetectionml


In [4]:
from azureml.core import Environment

# Retrieve the custom environment
environment = Environment.get(workspace=ws, name="lightgbm-environment")
print("Environment retrieved:", environment.name)

Environment retrieved: lightgbm-environment


In [6]:
import os
print("Current working directory:", os.getcwd())
print("Files in directory:", os.listdir())

Current working directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/fraud-detection-compute/code/Users/2kemo2sabe
Files in directory: ['.amlignore', '.amlignore.amltmp', '.ipynb_aml_checkpoints', 'CreateaComputeTargetipynb.ipynb', 'createacomputetargetipynb.ipynb.amltmp', 'creditcard.csv', 'Data_Preprocessing.ipynb', 'data_preprocessing.ipynb.amltmp', 'Deployment_model.ipynb', 'deployment_model.ipynb.amltmp', 'Hyperparameter_Tuning.ipynb', 'hyperparameter_tuning.ipynb.amltmp', 'lightgbm.ipynb.amltmp', 'LightGBM_model.ipynb', 'lightgbm_model.ipynb.amltmp', 'lightgbm_model.pkl', 'logistic_regression_model.pkl', 'score.py', 'score.py.amltmp', 'train_lightgbm.py', 'train_lightgbm.py.amltmp', 'xgboost.ipynb.amltmp', 'XGBoost_model.ipynb', 'xgboost_model.ipynb.amltmp', 'xgboost_model.pkl', 'X_test.csv', 'X_train.csv', 'y_test.csv', 'y_train.csv']


In [10]:
pip install --upgrade azureml-sdk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 685.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 15.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.7/313.7 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 29.8 MB/s eta 0:00:00
  Attempting uninstall: azureml-train-restclients-hyperdrive
    Found existing installation: azureml-train-restclients-hyperdrive 1.57.0
    Uninstalling azureml-train-restclients-hyperdrive-1.57.0:
      Successfully uninstalled azureml-train-restclients-hyperdrive-1.57.0
  Attempting uninstall: azureml-telemetry
    Found existing installation: azureml-telemetry 1.57.0
    Uninstalling azureml-telemetry-1.57.0:
      Successfully uninstalled azureml-telemetry-1.57.0
  Attempting uninstall: azureml-pipeline-core
    Found existing installation: azureml-pipeline-core 1.5

In [11]:
import azureml.core
print("Azure ML SDK Version:", azureml.core.VERSION)

Azure ML SDK Version: 1.59.0


In [27]:
from azureml.core import Workspace, Environment, Model, ComputeTarget
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.exceptions import ComputeTargetException

# Load the workspace
try:
    ws = Workspace.from_config()
    print("Workspace loaded:", ws.name)
except Exception as e:
    print("Failed to load workspace:", str(e))
    raise

# Retrieve the custom environment
try:
    environment = Environment.get(workspace=ws, name="lightgbm-environment")
    print("Environment retrieved:", environment.name)
except Exception as e:
    print("Failed to retrieve environment:", str(e))
    raise

# Define the AKS compute target name
aks_name = "aks-cluster"

# Check if the AKS compute target already exists
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print("AKS compute target already exists:", aks_name)
except ComputeTargetException:
    print("Creating a new AKS compute target...")

    # Define the AKS configuration
    aks_config = AksCompute.provisioning_configuration(
        vm_size="Standard_D2_v2",  # Use a larger VM size
        agent_count=3,  # Number of nodes in the cluster
        location="eastus"  # Replace with your preferred region
    )

    # Create the AKS compute target
    try:
        aks_target = ComputeTarget.create(workspace=ws, name=aks_name, provisioning_configuration=aks_config)
        aks_target.wait_for_completion(show_output=True)
    except Exception as e:
        print("Failed to create AKS compute target:", str(e))
        raise

# Define the inference configuration
try:
    inference_config = InferenceConfig(
        entry_script="score.py",  # Script to handle incoming requests
        environment=environment  # Use the custom environment
    )
    print("Inference configuration created.")
except Exception as e:
    print("Failed to create inference configuration:", str(e))
    raise

# Define the deployment configuration
try:
    aks_config = AksWebservice.deploy_configuration(
        cpu_cores=0.5,  # Reduce CPU request
        memory_gb=1,  # Reduce memory request
        description="Fraud Detection API"
    )
    print("Deployment configuration created.")
except Exception as e:
    print("Failed to create deployment configuration:", str(e))
    raise

# Deploy the model
try:
    aks_service = Model.deploy(
        workspace=ws,
        name="fraud-detection-api",
        models=[Model(ws, "lightgbm-fraud-detection")],  # Use the registered model
        inference_config=inference_config,
        deployment_config=aks_config,
        deployment_target=aks_target  # Use the AKS compute target
    )
    aks_service.wait_for_deployment(show_output=True)
    print("Deployment completed. Service URL:", aks_service.scoring_uri)
except Exception as e:
    print("Failed to deploy model:", str(e))
    raise

Workspace loaded: frauddetectionml
Environment retrieved: lightgbm-environment
Creating a new AKS compute target...
InProgress.............................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Inference configuration created.
Deployment configuration created.
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-02-08 23:29:06+00:00 Creating Container Registry if not exists..
2025-02-08 23:29:10+00:00 Use the existing image..
2025-02-08 23:29:19+00:00 Creating resources in AKS..
2025-02-08 23:29:28+00:00 Submitting deployment to compute.
2025-02-08 23:29:29+00:00 Checking the status of deployment fraud-detection-api..
2025-02-08 23:31:35+00:00 Checking the status of inference endpoint fraud-detection-api.
Succeed

/tmp/ipykernel_3152/2096725056.py:73: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  aks_service = Model.deploy(


In [28]:
from azureml.core.webservice import AksWebservice

# Retrieve the deployed service
aks_service = AksWebservice(workspace=ws, name="fraud-detection-api")

# Get the authentication key
auth_key = aks_service.get_keys()[0]  # Use the primary key
print("Authentication Key:", auth_key)

Authentication Key: RRClXaXfnIaNKqtafoYhQPALQVp3j9mo


In [30]:
import requests
import json

# Service URL
service_url = "http://48.217.133.143:80/api/v1/service/fraud-detection-api/score"

# Sample input data (replace with actual data)
sample_input = {
    "data": [
        {
            "Time": 0.0,
            "V1": -1.359807,
            "V2": -0.072781,
            "V3": 2.536347,
            "V4": 1.378155,
            "V5": -0.338321,
            "V6": 0.462388,
            "V7": 0.239599,
            "V8": 0.098698,
            "V9": 0.363787,
            "V10": 0.090794,
            "V11": -0.551600,
            "V12": -0.617801,
            "V13": -0.991390,
            "V14": -0.311169,
            "V15": 1.468177,
            "V16": -0.470401,
            "V17": 0.207971,
            "V18": 0.025791,
            "V19": 0.403993,
            "V20": 0.251412,
            "V21": -0.018307,
            "V22": 0.277838,
            "V23": -0.110474,
            "V24": 0.066928,
            "V25": 0.128539,
            "V26": -0.189115,
            "V27": 0.133558,
            "V28": -0.021053,
            "Amount": 149.62
        }
    ]
}

# Convert input data to JSON
input_data = json.dumps(sample_input)

# Set headers with authentication key
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {auth_key}"  # Include the authentication key
}

# Send request to the API
response = requests.post(service_url, input_data, headers=headers)

# Print the response
print("Response Status Code:", response.status_code)
print("Prediction Result:", response.json())

Response Status Code: 200
Prediction Result: {"predictions": [0]}


In [ ]:
import os

# Print current working directory
print("Current working directory:", os.getcwd())

# List files in the current working directory
print("Files in the current working directory:", os.listdir())